# Binary Classification Model Comparison

This notebook explores and compares multiple machine learning methods for a **binary classification problem** using ranked *League of Legends* solo/duo match data.

The primary goals of this analysis are:
- To **train and evaluate** a variety of classification models
- To **identify top-performing models** based on validation performance
- To **verify robustness** using both **shuffle testing** and **k-fold cross-validation**

The notebook includes:
- Data preprocessing and feature selection
- Implementation of various classification models (e.g., logistic regression, ensemble methods, neural networks)
- Model evaluation and comparison
- Post-model validation using shuffle tests and k-fold cross-validation

The final result is a selection of the best-performing models, supported by rigorous validation to assess generalization performance.

*Prepared by Barrett James McDonald | PhD Student | University of South Florida*

> **Reproducibility Note:**  
> All stochastic methods in this notebook (data splits, shuffles, and models) use `random_state=12` for reproducibility.  

In [1]:
#python libraries
import numpy as np
import pandas as pd
import numpy.linalg as LA

#data preprocessing & splitting
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

#dimensionality reduction
from sklearn.decomposition import PCA

#classification models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

#gradient boosting models
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

#evaluation metrics
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

## 1. Data Preprocessing and Subsampling

This section loads the raw match data and performs preprocessing to prepare the dataset for modeling. Specifically, it:

- Filters for ranked solo/duo *CLASSIC* games only,
- Removes irrelevant metadata (e.g., match IDs, timestamps),
- Excludes player history and champion mastery statistics,
- Drops categorical identifiers such as `summoner_id` and `champion_id`,
- Retains only numeric, interpretable performance metrics.

A random subsample of 2,500 observations is selected to enable faster experimentation and model comparison.

In [2]:
# Load the data
df_csv = pd.read_csv("league_data.csv", dtype={'win': str})

# Drop metadata/player history columns
columns_to_drop = [
    # Identifiers
    'game_id', 'game_version', 'participant_id', 'puuid', 'summoner_name', 'summoner_id',
    
    # Ranked data (solo/flex)
    'solo_tier', 'solo_rank', 'solo_lp', 'solo_wins', 'solo_losses',
    'flex_tier', 'flex_rank', 'flex_lp', 'flex_wins', 'flex_losses',

    # Champion mastery / history
    'champion_mastery_level', 'champion_mastery_points',
    'champion_mastery_pointsSinceLastLevel', 'champion_mastery_pointsUntilNextLevel',
    'champion_mastery_tokensEarned',
    'champion_mastery_lastPlayTime', 'champion_mastery_lastPlayTime_utc',
    'summoner_level',

    # Match metadata
    'champion_id', 'map_id', 'platform_id', 'game_type', 'team_id',
    'game_start_utc', 'queue_id', 'game_mode',
]

# Filter for CLASSIC + ranked solo/duo games
df_filtered = df_csv[(df_csv['game_mode'] == 'CLASSIC') & (df_csv['queue_id'] == 420)].copy()

# Drop metadata columns
df_filtered_cleaned = df_filtered.drop(columns=[col for col in columns_to_drop if col in df_filtered.columns])

# Convert 'win' column to binary
df_filtered_cleaned['win'] = (df_filtered_cleaned['win'] == 'TRUE').astype(int)

# Drop non-numeric/categorical columns (and item columns)
df_numeric_only = df_filtered_cleaned.drop(columns=df_filtered_cleaned.select_dtypes(include=['object', 'category']).columns)
df_numeric_only = df_numeric_only.drop(columns=[col for col in df_numeric_only.columns if col.startswith("item")])

# Final predictor/response matrices
X = df_numeric_only.drop(columns=['win']).fillna(df_numeric_only.mean())
y = df_numeric_only['win']

# --- Subsample preparation (2,500 observations) ---
#for reproducibility, set the numpy random seed to 12
np.random.seed(12)
sample_indices = np.random.choice(X.index, size=2500, replace=False)
X_sample = X.loc[sample_indices]
y_sample = y.loc[sample_indices]

# --- Show final features being used ---
print(f"Total features used: {X.shape[1]}")
print("List of final features:")
print(X.columns.tolist())

Total features used: 49
List of final features:
['game_duration', 'kills', 'deaths', 'assists', 'baron_kills', 'dragon_kills', 'gold_earned', 'gold_spent', 'total_damage_dealt', 'total_damage_dealt_to_champions', 'physical_damage_dealt_to_champions', 'magic_damage_dealt_to_champions', 'true_damage_dealt_to_champions', 'damage_dealt_to_objectives', 'damage_dealt_to_turrets', 'total_damage_taken', 'physical_damage_taken', 'magic_damage_taken', 'true_damage_taken', 'time_ccing_others', 'vision_score', 'wards_placed', 'wards_killed', 'vision_wards_bought_in_game', 'final_abilityHaste', 'final_abilityPower', 'final_armor', 'final_armorPen', 'final_armorPenPercent', 'final_attackDamage', 'final_attackSpeed', 'final_bonusArmorPenPercent', 'final_bonusMagicPenPercent', 'final_ccReduction', 'final_cooldownReduction', 'final_health', 'final_healthMax', 'final_healthRegen', 'final_lifesteal', 'final_magicPen', 'final_magicPenPercent', 'final_magicResist', 'final_movementSpeed', 'final_omnivamp', 

## 2. Robust PCA + Logistic Regression

This section uses a manually implemented **Robust PCA** to decompose the scaled predictor matrix into low-rank and sparse components. The low-rank matrix is used for further dimensionality reduction via PCA before fitting a logistic regression classifier.

Performance metrics (Accuracy, F1 Score, ROC AUC) are computed on a held-out test set.

In [3]:
# --- Defining Robust PCA Manual Calculation of L and S ---
def robust_pca_fast(M, max_iter=150, tol=1e-4):
    """
    Perform Robust Principal Component Analysis (RPCA) on matrix M.
    Decomposes M into L (low-rank) and S (sparse) components using
    the Principal Component Pursuit algorithm.

    Args:
        M (np.ndarray): Input data matrix (rows = observations, cols = features)
        max_iter (int): Maximum number of iterations
        tol (float): Convergence tolerance (Frobenius norm of residual)

    Returns:
        L (np.ndarray): Low-rank matrix
        S (np.ndarray): Sparse matrix
    """
    
    # --- Helper function: soft-thresholding operator ---
    def shrinkage_operator(x, tau):
        # Applies soft-thresholding elementwise
        return np.sign(x) * np.maximum(np.abs(x) - tau, 0.)

    # --- Helper function: thresholded SVD ---
    def svd_thresholding_operator(X, tau):
        # Applies singular value thresholding to keep only large singular values
        U, S, Vh = LA.svd(X, full_matrices=False)
        S_thresh = shrinkage_operator(S, tau)
        return U @ np.diag(S_thresh) @ Vh

    # --- Initialization ---
    S = np.zeros_like(M)              # Start with zero sparse matrix
    Y = np.zeros_like(M)              # Lagrange multiplier (dual variable)
    mu = np.prod(M.shape) / (4.0 * LA.norm(M, ord=1))  # Step size parameter
    mu_inv = 1.0 / mu
    lam = 1.0 / np.sqrt(np.max(M.shape))               # Regularization parameter

    for _ in range(max_iter):
        # --- Low-rank update via SVD thresholding ---
        L = svd_thresholding_operator(M - S + mu_inv * Y, mu_inv)

        # --- Sparse matrix update via elementwise shrinkage ---
        S = shrinkage_operator(M - L + mu_inv * Y, lam * mu_inv)

        # --- Dual variable update (Lagrange multiplier) ---
        Y = Y + mu * (M - L - S)

        # --- Check convergence ---
        error = LA.norm(M - L - S, ord='fro')
        if error < tol:
            break

    return L, S
    
# --- Preprocessing and robust PCA ---
scaler_raw = StandardScaler()
X_scaled_sample = scaler_raw.fit_transform(X_sample)
L_sample, S_sample = robust_pca_fast(X_scaled_sample)

# --- Train/test split and scale ---
X_train, X_test, y_train, y_test = train_test_split(
    L_sample, y_sample, test_size=0.3, stratify=y_sample, random_state=12
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- PCA: find n_components for >=80% variance ---
pca_full = PCA(random_state=12)
pca_full.fit(X_train_scaled)
cumulative_variance = np.cumsum(pca_full.explained_variance_ratio_)
n_components_80 = np.searchsorted(cumulative_variance, 0.80) + 1  # +1 because index starts at 0

print(f"Number of PCs for at least 80% variance: {n_components_80}")

# --- Fit PCA with this n_components ---
pca = PCA(n_components=n_components_80, random_state=12)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# --- Logistic Regression ---
log_reg = LogisticRegression(max_iter=1000, random_state=12)
log_reg.fit(X_train_pca, y_train)
y_pred = log_reg.predict(X_test_pca)
y_proba = log_reg.predict_proba(X_test_pca)[:, 1]

# --- Performance output ---
print(f"PCLR (Robust PCA + Logistic Regression) with {n_components_80} PCs (>=80% variance) (2500 row subsample):")
print(f"Accuracy:  {accuracy_score(y_test, y_pred):.4f}")
print(f"F1 Score:  {f1_score(y_test, y_pred):.4f}")
print(f"ROC AUC:   {roc_auc_score(y_test, y_proba):.4f}")

Number of PCs for at least 80% variance: 15
PCLR (Robust PCA + Logistic Regression) with 15 PCs (>=80% variance) (2500 row subsample):
Accuracy:  0.8120
F1 Score:  0.8107
ROC AUC:   0.9104


## 3. Logistic Regression (No PCA): L1 vs L2 Regularization

This section evaluates logistic regression models trained directly on the full set of numeric features (no dimensionality reduction).

Two forms of regularization are compared:
- **L2 (Ridge):** Penalizes the squared magnitude of coefficients. Tends to shrink coefficients uniformly but keeps all features.
- **L1 (Lasso):** Penalizes the absolute value of coefficients. Can set some coefficients exactly to zero, thus performing feature selection.

We apply both penalties to the same training/test split of a 2,500-observation subsample and evaluate their performance.

In [4]:
# --- Split and scale ---
X_train, X_test, y_train, y_test = train_test_split(
    X_sample, y_sample, test_size=0.3, stratify=y_sample, random_state=12
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- L2 (Ridge) Regularization ---
log_reg_l2 = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=1000, random_state=12)
log_reg_l2.fit(X_train_scaled, y_train)
y_pred_l2 = log_reg_l2.predict(X_test_scaled)
y_proba_l2 = log_reg_l2.predict_proba(X_test_scaled)[:, 1]

# --- L1 (Lasso) Regularization ---
log_reg_l1 = LogisticRegression(penalty='l1', solver='liblinear', max_iter=1000, random_state=12)
log_reg_l1.fit(X_train_scaled, y_train)
y_pred_l1 = log_reg_l1.predict(X_test_scaled)
y_proba_l1 = log_reg_l1.predict_proba(X_test_scaled)[:, 1]

# --- Print results ---
print("Logistic Regression without PCA:")

print("L2 Regularization (2500 row subsample):")
print(f"Accuracy:  {accuracy_score(y_test, y_pred_l2):.4f}")
print(f"F1 Score:  {f1_score(y_test, y_pred_l2):.4f}")
print(f"ROC AUC:   {roc_auc_score(y_test, y_proba_l2):.4f}\n")

print("L1 Regularization (2500 row subsample):")
print(f"Accuracy:  {accuracy_score(y_test, y_pred_l1):.4f}")
print(f"F1 Score:  {f1_score(y_test, y_pred_l1):.4f}")
print(f"ROC AUC:   {roc_auc_score(y_test, y_proba_l1):.4f}")

Logistic Regression without PCA:
L2 Regularization (2500 row subsample):
Accuracy:  0.8853
F1 Score:  0.8868
ROC AUC:   0.9464

L1 Regularization (2500 row subsample):
Accuracy:  0.8840
F1 Score:  0.8854
ROC AUC:   0.9464


## 4. Decision Tree Classifier

This section implements a basic **Decision Tree Classifier** trained on the same scaled numeric data used in previous models.

Decision Trees are intuitive and interpretable models that recursively split the feature space to classify observations. While they tend to overfit without pruning or regularization, they offer a useful baseline for tree-based methods like Random Forest and Gradient Boosted Trees.

We evaluate the model on a 70/30 train-test split and report standard classification metrics.

In [5]:
# --- Train/test split and scale ---
X_train, X_test, y_train, y_test = train_test_split(
    X_sample, y_sample, test_size=0.3, stratify=y_sample, random_state=12
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Train Decision Tree ---
tree_clf = DecisionTreeClassifier(random_state=12)
tree_clf.fit(X_train_scaled, y_train)
y_pred_tree = tree_clf.predict(X_test_scaled)
y_proba_tree = tree_clf.predict_proba(X_test_scaled)[:, 1]

# --- Evaluate performance ---
print("Decision Tree Results (2500 row subsample):")
print(f"Accuracy:  {accuracy_score(y_test, y_pred_tree):.4f}")
print(f"F1 Score:  {f1_score(y_test, y_pred_tree):.4f}")
print(f"ROC AUC:   {roc_auc_score(y_test, y_proba_tree):.4f}")

Decision Tree Results (2500 row subsample):
Accuracy:  0.7800
F1 Score:  0.7930
ROC AUC:   0.7789


## 5. Random Forest Classifier

This section applies a **Random Forest**, an ensemble learning method that builds a collection of decision trees and combines their predictions to improve generalization.

Random Forests reduce overfitting by:
- Training each tree on a random bootstrap sample of the data
- Using a random subset of features at each split

Here, we train a forest of 100 trees on a 70/30 split of the scaled data and report accuracy, F1 score, and ROC AUC. This provides a more robust baseline than a single decision tree.

In [6]:
# --- Train/test split and scale ---
X_train, X_test, y_train, y_test = train_test_split(
    X_sample, y_sample, test_size=0.3, stratify=y_sample, random_state=12
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Train Random Forest ---
forest_clf = RandomForestClassifier(n_estimators=100, random_state=12)
forest_clf.fit(X_train_scaled, y_train)
y_pred_forest = forest_clf.predict(X_test_scaled)
y_proba_forest = forest_clf.predict_proba(X_test_scaled)[:, 1]

# --- Evaluate performance ---
print("Random Forest Results (2500 row subsample):")
print(f"Accuracy:  {accuracy_score(y_test, y_pred_forest):.4f}")
print(f"F1 Score:  {f1_score(y_test, y_pred_forest):.4f}")
print(f"ROC AUC:   {roc_auc_score(y_test, y_proba_forest):.4f}")

Random Forest Results (2500 row subsample):
Accuracy:  0.8667
F1 Score:  0.8708
ROC AUC:   0.9362


## 6. XGBoost Classifier

This section trains an **Extreme Gradient Boosting (XGBoost)** classifier, a high-performance ensemble method known for its ability to handle:
- Imbalanced classes
- Nonlinear feature interactions
- Feature importance and missing data

XGBoost builds trees sequentially, where each new tree tries to correct the errors of the previous one, minimizing a specified loss function—in this case, **log loss**.

We train the model on scaled data with default hyperparameters and evaluate using accuracy, F1 score, and ROC AUC.

In [7]:
# --- Train/test split and scale ---
X_train, X_test, y_train, y_test = train_test_split(
    X_sample, y_sample, test_size=0.3, stratify=y_sample, random_state=12
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Train XGBoost ---
xgb_clf = XGBClassifier(eval_metric='logloss', random_state=12)
xgb_clf.fit(X_train_scaled, y_train)
y_pred_xgb = xgb_clf.predict(X_test_scaled)
y_proba_xgb = xgb_clf.predict_proba(X_test_scaled)[:, 1]

# --- Evaluate performance ---
print("XGBoost Results (2500 row subsample):")
print(f"Accuracy:  {accuracy_score(y_test, y_pred_xgb):.4f}")
print(f"F1 Score:  {f1_score(y_test, y_pred_xgb):.4f}")
print(f"ROC AUC:   {roc_auc_score(y_test, y_proba_xgb):.4f}")

XGBoost Results (2500 row subsample):
Accuracy:  0.8667
F1 Score:  0.8718
ROC AUC:   0.9373


## 7. LightGBM Classifier

This section fits a **LightGBM (Light Gradient Boosting Machine)** model—an efficient gradient boosting framework that uses histogram-based algorithms for faster training and lower memory usage.

Compared to XGBoost, LightGBM is:
- Faster on large datasets with many features
- Capable of handling categorical variables natively (though we use numeric-only data here)
- Often just as accurate (or better) with less tuning

We train LightGBM on a 70/30 train-test split of scaled data, evaluating its predictive performance with standard classification metrics.

In [8]:
# --- Train/test split and scale ---
X_train, X_test, y_train, y_test = train_test_split(
    X_sample, y_sample, test_size=0.3, stratify=y_sample, random_state=12
)
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)
X_test_scaled = pd.DataFrame(
    scaler.transform(X_test),
    columns=X_test.columns,
    index=X_test.index
)

# --- Train LightGBM ---
lgbm_clf = LGBMClassifier(verbose=-1, random_state=12)
lgbm_clf.fit(X_train_scaled, y_train)
y_pred_lgbm = lgbm_clf.predict(X_test_scaled)
y_proba_lgbm = lgbm_clf.predict_proba(X_test_scaled)[:, 1]

# --- Evaluate performance ---
print("LightGBM Results (2500 row subsample):")
print(f"Accuracy:  {accuracy_score(y_test, y_pred_lgbm):.4f}")
print(f"F1 Score:  {f1_score(y_test, y_pred_lgbm):.4f}")
print(f"ROC AUC:   {roc_auc_score(y_test, y_proba_lgbm):.4f}")

LightGBM Results (2500 row subsample):
Accuracy:  0.8773
F1 Score:  0.8821
ROC AUC:   0.9429


## 8. Support Vector Machine (RBF Kernel)

This section trains a **Support Vector Machine (SVM)** with a radial basis function (RBF) kernel. SVMs aim to find the optimal hyperplane that separates classes with the **maximum margin** in a high-dimensional space.

Key notes:
- The **RBF kernel** allows the model to learn nonlinear decision boundaries by implicitly mapping features into a higher-dimensional space.
- We set `probability=True` to enable **probability estimates**, which are required for computing the **ROC AUC** score.

Although SVMs can be computationally intensive, especially on large datasets, they often perform well with clean, scaled features.

In [9]:
# --- Train/test split and scale ---
X_train, X_test, y_train, y_test = train_test_split(
    X_sample, y_sample, test_size=0.3, stratify=y_sample, random_state=12
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Train SVM (with probability enabled for ROC AUC) ---
svm_clf = SVC(probability=True, kernel='rbf', random_state=12)
svm_clf.fit(X_train_scaled, y_train)
y_pred_svm = svm_clf.predict(X_test_scaled)
y_proba_svm = svm_clf.predict_proba(X_test_scaled)[:, 1]

# --- Evaluate performance ---
print("Support Vector Machine Results (2500 row subsample):")
print(f"Accuracy:  {accuracy_score(y_test, y_pred_svm):.4f}")
print(f"F1 Score:  {f1_score(y_test, y_pred_svm):.4f}")
print(f"ROC AUC:   {roc_auc_score(y_test, y_proba_svm):.4f}")

Support Vector Machine Results (2500 row subsample):
Accuracy:  0.8667
F1 Score:  0.8724
ROC AUC:   0.9375


## 9. K-Nearest Neighbors (KNN)

This section applies the **K-Nearest Neighbors (KNN)** algorithm using \( k = 5 \). KNN is a **non-parametric** method that classifies a sample based on the majority label among its nearest neighbors in feature space.

Key characteristics:
- Simple, interpretable, and effective with well-scaled, low-dimensional data
- Sensitive to irrelevant features and class imbalance
- Performance depends heavily on the choice of **k** and the distance metric

Here, we standardize features and evaluate KNN on a 70/30 split of a 2,500-observation subsample.

In [10]:
# --- Train/test split and scale ---
X_train, X_test, y_train, y_test = train_test_split(
    X_sample, y_sample, test_size=0.3, stratify=y_sample, random_state=12
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Train KNN ---
knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_train_scaled, y_train)
y_pred_knn = knn_clf.predict(X_test_scaled)
y_proba_knn = knn_clf.predict_proba(X_test_scaled)[:, 1]

# --- Evaluate performance ---
print("K-Nearest Neighbors Results (2500 row subsample):")
print(f"Accuracy:  {accuracy_score(y_test, y_pred_knn):.4f}")
print(f"F1 Score:  {f1_score(y_test, y_pred_knn):.4f}")
print(f"ROC AUC:   {roc_auc_score(y_test, y_proba_knn):.4f}")

K-Nearest Neighbors Results (2500 row subsample):
Accuracy:  0.7920
F1 Score:  0.7845
ROC AUC:   0.8681


## 10. Naive Bayes Classifier

This section implements a **Naive Bayes classifier**, specifically using the **GaussianNB** variant. Naive Bayes is a probabilistic model based on Bayes’ Theorem, assuming **feature independence** given the class label.

Why it matters:
- Surprisingly effective in high-dimensional settings
- Fast to train and easy to interpret
- Works best when features are roughly independent (which is rare, but doesn’t always hurt performance)

We fit the model on scaled data and evaluate using accuracy, F1 score, and ROC AUC.

In [11]:
# --- Train/test split and scale ---
X_train, X_test, y_train, y_test = train_test_split(
    X_sample, y_sample, test_size=0.3, stratify=y_sample, random_state=12
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Train Naive Bayes ---
nb_clf = GaussianNB()
nb_clf.fit(X_train_scaled, y_train)
y_pred_nb = nb_clf.predict(X_test_scaled)
y_proba_nb = nb_clf.predict_proba(X_test_scaled)[:, 1]

# --- Evaluate performance ---
print("Naive Bayes Results (2500 row subsample):")
print(f"Accuracy:  {accuracy_score(y_test, y_pred_nb):.4f}")
print(f"F1 Score:  {f1_score(y_test, y_pred_nb):.4f}")
print(f"ROC AUC:   {roc_auc_score(y_test, y_proba_nb):.4f}")

Naive Bayes Results (2500 row subsample):
Accuracy:  0.7267
F1 Score:  0.7076
ROC AUC:   0.8327


## 11. Neural Network: Multi-Layer Perceptron (MLP)

This final model is a **Neural Network**, specifically a **Multi-Layer Perceptron (MLP)**. MLPs are **feedforward neural networks** that learn complex, nonlinear relationships through layers of interconnected neurons.

Model architecture:
- One hidden layer with 100 neurons
- Uses ReLU activation (default)
- Optimized with the Adam solver
- Trained for up to 500 iterations (or until convergence)

While MLPs often require more tuning and training time, they can outperform traditional models when the data contains deep, abstract patterns.

In [12]:
# --- Train/test split and scale ---
X_train, X_test, y_train, y_test = train_test_split(
    X_sample, y_sample, test_size=0.3, stratify=y_sample, random_state=12
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Train Neural Network (MLP) ---
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=12)
mlp_clf.fit(X_train_scaled, y_train)
y_pred_mlp = mlp_clf.predict(X_test_scaled)
y_proba_mlp = mlp_clf.predict_proba(X_test_scaled)[:, 1]

# --- Evaluate performance ---
print("Neural Network (MLP) Results (2500 row subsample):")
print(f"Accuracy:  {accuracy_score(y_test, y_pred_mlp):.4f}")
print(f"F1 Score:  {f1_score(y_test, y_pred_mlp):.4f}")
print(f"ROC AUC:   {roc_auc_score(y_test, y_proba_mlp):.4f}")

Neural Network (MLP) Results (2500 row subsample):
Accuracy:  0.8693
F1 Score:  0.8731
ROC AUC:   0.9440


## Note on Model Selection:
After evaluating all candidate models on subsampled data, we identified the five models with the strongest validation performance (based on ROC AUC, F1 score, and accuracy).  

To provide a robust assessment of their true predictive capabilities, we now retrain and evaluate these top five models on the entire, cleaned dataset.  
 
This ensures that our final comparisons reflect each model's ability to generalize across the full feature space and observation set, and forms the basis for the results reported in the main tables of this study.

## 12. Tuned Neural Network (MLP) on Full Dataset

In this model, we revisit the **Multi-Layer Perceptron (MLP)** and apply **hyperparameter tuning** for improved performance. Key changes include:

- **Architecture:** Two hidden layers with 128 and 64 neurons
- **Activation:** ReLU (Rectified Linear Unit)
- **Optimizer:** Adam (adaptive learning rate)
- **Training:** 1,000 epochs (or until convergence)
- **Dataset:** Full cleaned dataset (not a subsample)

These changes aim to increase the model's capacity to learn complex, nonlinear relationships across the full feature space.

In [13]:
# --- Full dataset (already preprocessed into X and y) ---
X_full = X.copy()
y_full = y.copy()

# --- Train/test split and scale ---
X_train, X_test, y_train, y_test = train_test_split(
    X_full, y_full, test_size=0.3, stratify=y_full, random_state=12
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Tuned MLP Neural Network ---
mlp_tuned = MLPClassifier(
    hidden_layer_sizes=(128, 64),
    activation='relu',
    solver='adam',
    max_iter=1000,
    random_state=12
)
mlp_tuned.fit(X_train_scaled, y_train)
y_pred_mlp = mlp_tuned.predict(X_test_scaled)
y_proba_mlp = mlp_tuned.predict_proba(X_test_scaled)[:, 1]

# --- Evaluate performance ---
print("Tuned Neural Network Results (Full Dataset):")
print(f"Accuracy:  {accuracy_score(y_test, y_pred_mlp):.4f}")
print(f"F1 Score:  {f1_score(y_test, y_pred_mlp):.4f}")
print(f"ROC AUC:   {roc_auc_score(y_test, y_proba_mlp):.4f}")

Tuned Neural Network Results (Full Dataset):
Accuracy:  0.8774
F1 Score:  0.8771
ROC AUC:   0.9481


## 13. LightGBM on Full Dataset

This model revisits **LightGBM**, this time training on the entire preprocessed dataset rather than a subsample.

By scaling and fitting LightGBM on the full data, we aim to:
- Leverage more information for training
- Capture rarer patterns that may not be present in smaller subsamples
- Potentially improve performance and stability

We retain feature names by converting the scaled arrays back into DataFrames, which can help with future interpretability and feature importance analysis.

In [14]:
# --- Full dataset (already preprocessed into X and y) ---
X_full = X.copy()
y_full = y.copy()

# --- Train/test split and scale ---
X_train, X_test, y_train, y_test = train_test_split(
    X_full, y_full, test_size=0.3, stratify=y_full, random_state=12
)
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

# --- Train LightGBM ---
lgbm_clf = LGBMClassifier(verbose=-1, random_state=12)
lgbm_clf.fit(X_train_scaled, y_train)
y_pred_lgbm = lgbm_clf.predict(X_test_scaled)
y_proba_lgbm = lgbm_clf.predict_proba(X_test_scaled)[:, 1]

# --- Evaluate performance ---
print("LightGBM Results (Full Dataset):")
print(f"Accuracy:  {accuracy_score(y_test, y_pred_lgbm):.4f}")
print(f"F1 Score:  {f1_score(y_test, y_pred_lgbm):.4f}")
print(f"ROC AUC:   {roc_auc_score(y_test, y_proba_lgbm):.4f}")

LightGBM Results (Full Dataset):
Accuracy:  0.8958
F1 Score:  0.8956
ROC AUC:   0.9648


## 14. XGBoost on Full Dataset

This section returns to **XGBoost**, but this time we train on the entire preprocessed dataset instead of a subsample.

Key details:
- Full dataset provides a richer training signal, potentially capturing subtler relationships
- XGBoost is configured to minimize **log loss**, which is appropriate for binary classification with probabilistic outputs
- Data is standardized before training, though XGBoost can handle unscaled input—scaling helps ensure fair comparison across models

We evaluate model performance using accuracy, F1 score, and ROC AUC.

In [15]:
# --- Full dataset (already preprocessed into X and y) ---
X_full = X.copy()
y_full = y.copy()

# --- Train/test split and scale ---
X_train, X_test, y_train, y_test = train_test_split(
    X_full, y_full, test_size=0.3, stratify=y_full, random_state=12
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Train XGBoost ---
xgb_clf = XGBClassifier(eval_metric='logloss', random_state=12)
xgb_clf.fit(X_train_scaled, y_train)
y_pred_xgb = xgb_clf.predict(X_test_scaled)
y_proba_xgb = xgb_clf.predict_proba(X_test_scaled)[:, 1]

# --- Evaluate performance ---
print("XGBoost Results (Full Dataset):")
print(f"Accuracy:  {accuracy_score(y_test, y_pred_xgb):.4f}")
print(f"F1 Score:  {f1_score(y_test, y_pred_xgb):.4f}")
print(f"ROC AUC:   {roc_auc_score(y_test, y_proba_xgb):.4f}")

XGBoost Results (Full Dataset):
Accuracy:  0.8949
F1 Score:  0.8948
ROC AUC:   0.9640


## 15. L1-Regularized Logistic Regression on Full Dataset

Here we train a **logistic regression model with L1 (Lasso) regularization** on the full dataset.

L1 regularization:
- Encourages sparsity in the model coefficients (i.e., some become exactly zero)
- Effectively performs **feature selection** by shrinking less informative features
- Helps prevent overfitting when many features are present

We scale the full dataset before training, and evaluate using accuracy, F1 score, and ROC AUC.

In [16]:
# --- Full dataset (already preprocessed into X and y) ---
X_full = X.copy()
y_full = y.copy()

# --- Train/test split and scale ---
X_train, X_test, y_train, y_test = train_test_split(
    X_full, y_full, test_size=0.3, stratify=y_full, random_state=12
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- L1 (Lasso) Logistic Regression ---
log_reg_l1 = LogisticRegression(
    penalty='l1', solver='liblinear', max_iter=1000, random_state=12
)
log_reg_l1.fit(X_train_scaled, y_train)
y_pred_l1 = log_reg_l1.predict(X_test_scaled)
y_proba_l1 = log_reg_l1.predict_proba(X_test_scaled)[:, 1]

# --- Evaluate performance ---
print("L1 Logistic Regression Results (Full Dataset):")
print(f"Accuracy:  {accuracy_score(y_test, y_pred_l1):.4f}")
print(f"F1 Score:  {f1_score(y_test, y_pred_l1):.4f}")
print(f"ROC AUC:   {roc_auc_score(y_test, y_proba_l1):.4f}")

L1 Logistic Regression Results (Full Dataset):
Accuracy:  0.8827
F1 Score:  0.8810
ROC AUC:   0.9464


## 16. L2-Regularized Logistic Regression on Full Dataset

Here we train a **logistic regression model with L2 (Ridge) regularization** on the full dataset.

L2 regularization:
- Penalizes large coefficients, but does **not** enforce sparsity (all features usually retained)
- Tends to distribute weights more evenly among correlated features
- Helps prevent overfitting, especially when features are collinear or numerous

We scale the full dataset before training, and evaluate using accuracy, F1 score, and ROC AUC.

In [17]:
# --- Full dataset (already preprocessed into X and y) ---
X_full = X.copy()
y_full = y.copy()

# --- Train/test split and scale ---
X_train, X_test, y_train, y_test = train_test_split(
    X_full, y_full, test_size=0.3, stratify=y_full, random_state=12
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- L2 (Ridge) Logistic Regression ---
log_reg_l2 = LogisticRegression(
    penalty='l2', solver='lbfgs', max_iter=1000, random_state=12
)
log_reg_l2.fit(X_train_scaled, y_train)
y_pred_l2 = log_reg_l2.predict(X_test_scaled)
y_proba_l2 = log_reg_l2.predict_proba(X_test_scaled)[:, 1]

# --- Evaluate performance ---
print("L2 Logistic Regression Results (Full Dataset):")
print(f"Accuracy:  {accuracy_score(y_test, y_pred_l2):.4f}")
print(f"F1 Score:  {f1_score(y_test, y_pred_l2):.4f}")
print(f"ROC AUC:   {roc_auc_score(y_test, y_proba_l2):.4f}")

L2 Logistic Regression Results (Full Dataset):
Accuracy:  0.8822
F1 Score:  0.8806
ROC AUC:   0.9462


## Model Validation Strategy:

To ensure the reliability and generalizability of our findings, we perform **two complementary validation methods** for each top-performing model:  
 
**Shuffle tests:** For each model, we repeat the shuffle test multiple times. This involves randomly permuting the outcome labels and retraining the model, confirming that predictive performance drops to chance levels (mean ROC AUC, F1, and accuracy all ≈ 0.5). This rules out model bias, data leakage, or spurious relationships.
 
**K-fold cross-validation:** We use 5-fold stratified cross-validation to assess each model’s stability and predictive performance across different train/test splits. We report the mean and standard deviation of ROC AUC, along with average F1 score and accuracy across folds.
 
These validation steps provide strong evidence that our results are robust and not an artifact of data sampling or model overfitting.

## 16. Shuffle Test Results for LightGBM & XGBoost

The table below reports mean ROC AUC, F1 score, and accuracy for 10 shuffle runs.

In [18]:
n_runs = 10

lgbm_auc_shuff, lgbm_f1_shuff, lgbm_acc_shuff = [], [], []
xgb_auc_shuff, xgb_f1_shuff, xgb_acc_shuff = [], [], []

for i in range(n_runs):
    # --- Shuffle labels independently (different seed each time) ---
    y_shuffled = y_full.sample(frac=1, random_state=12 + i).reset_index(drop=True)
    X_shuffled = X_full.reset_index(drop=True)  # Align index with shuffled y

    # --- Train/test split and scale (on mismatched X/y) ---
    X_train, X_test, y_train_shuffled, y_test_shuffled = train_test_split(
        X_shuffled, y_shuffled, test_size=0.3, stratify=y_shuffled, random_state=12 + i
    )

    scaler = StandardScaler()
    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
    X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    # --- LightGBM on shuffled labels ---
    lgbm_clf = LGBMClassifier(verbose=-1, random_state=12)
    lgbm_clf.fit(X_train_scaled, y_train_shuffled)
    y_pred_lgbm = lgbm_clf.predict(X_test_scaled)
    y_proba_lgbm = lgbm_clf.predict_proba(X_test_scaled)[:, 1]
    lgbm_auc_shuff.append(roc_auc_score(y_test_shuffled, y_proba_lgbm))
    lgbm_f1_shuff.append(f1_score(y_test_shuffled, y_pred_lgbm))
    lgbm_acc_shuff.append(accuracy_score(y_test_shuffled, y_pred_lgbm))

    # --- XGBoost on shuffled labels ---
    xgb_clf = XGBClassifier(eval_metric='logloss', random_state=12)
    xgb_clf.fit(X_train_scaled, y_train_shuffled)
    y_pred_xgb = xgb_clf.predict(X_test_scaled)
    y_proba_xgb = xgb_clf.predict_proba(X_test_scaled)[:, 1]
    xgb_auc_shuff.append(roc_auc_score(y_test_shuffled, y_proba_xgb))
    xgb_f1_shuff.append(f1_score(y_test_shuffled, y_pred_xgb))
    xgb_acc_shuff.append(accuracy_score(y_test_shuffled, y_pred_xgb))

# --- Results ---
print("LightGBM (Shuffled Labels, 10 runs):")
print("AUCs:", np.round(lgbm_auc_shuff, 4))
print("F1s:", np.round(lgbm_f1_shuff, 4))
print("Accuracies:", np.round(lgbm_acc_shuff, 4))
print(f"Mean AUC: {np.mean(lgbm_auc_shuff):.4f} | Std Dev: {np.std(lgbm_auc_shuff):.4f}")
print(f"Mean F1: {np.mean(lgbm_f1_shuff):.4f}")
print(f"Mean Accuracy: {np.mean(lgbm_acc_shuff):.4f}\n")

print("XGBoost (Shuffled Labels, 10 runs):")
print("AUCs:", np.round(xgb_auc_shuff, 4))
print("F1s:", np.round(xgb_f1_shuff, 4))
print("Accuracies:", np.round(xgb_acc_shuff, 4))
print(f"Mean AUC: {np.mean(xgb_auc_shuff):.4f} | Std Dev: {np.std(xgb_auc_shuff):.4f}")
print(f"Mean F1: {np.mean(xgb_f1_shuff):.4f}")
print(f"Mean Accuracy: {np.mean(xgb_acc_shuff):.4f}")

LightGBM (Shuffled Labels, 10 runs):
AUCs: [0.5079 0.5113 0.5054 0.507  0.5028 0.5032 0.4919 0.4976 0.5041 0.5111]
F1s: [0.5074 0.5102 0.5049 0.4923 0.5036 0.5029 0.5005 0.4921 0.5085 0.5052]
Accuracies: [0.5106 0.5118 0.5027 0.5001 0.4973 0.4978 0.4957 0.4993 0.503  0.5086]
Mean AUC: 0.5042 | Std Dev: 0.0056
Mean F1: 0.5028
Mean Accuracy: 0.5027

XGBoost (Shuffled Labels, 10 runs):
AUCs: [0.5052 0.5128 0.4992 0.5047 0.4985 0.5081 0.4976 0.5008 0.5125 0.4992]
F1s: [0.5047 0.51   0.4952 0.5076 0.5052 0.5117 0.4962 0.5044 0.5035 0.4997]
Accuracies: [0.5037 0.5132 0.4993 0.5048 0.4969 0.5033 0.4955 0.5034 0.5078 0.4967]
Mean AUC: 0.5039 | Std Dev: 0.0054
Mean F1: 0.5038
Mean Accuracy: 0.5025


## 17. K-Fold Cross-Validation for LightGBM & XGBoost

The table below summarizes mean ROC AUC, F1 score, and accuracy across 5 folds for each model.

In [19]:
# --- Full dataset (already preprocessed into X and y) ---
X_full = X.copy()
y_full = y.copy()

# --- Scale full data ---
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_full)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)  # Retain column names

# --- 5-Fold Stratified CV ---
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=12)

lgbm_aucs, lgbm_f1s, lgbm_accs = [], [], []
xgb_aucs, xgb_f1s, xgb_accs = [], [], []

for train_index, test_index in kf.split(X_scaled_df, y_full):
    X_train, X_test = X_scaled_df.iloc[train_index], X_scaled_df.iloc[test_index]
    y_train, y_test = y_full.iloc[train_index], y_full.iloc[test_index]

    # LightGBM
    lgbm_model = LGBMClassifier(verbose=-1, random_state=12)
    lgbm_model.fit(X_train, y_train)
    lgbm_pred = lgbm_model.predict(X_test)
    lgbm_proba = lgbm_model.predict_proba(X_test)[:, 1]
    lgbm_auc = roc_auc_score(y_test, lgbm_proba)
    lgbm_f1 = f1_score(y_test, lgbm_pred)
    lgbm_acc = accuracy_score(y_test, lgbm_pred)
    lgbm_aucs.append(lgbm_auc)
    lgbm_f1s.append(lgbm_f1)
    lgbm_accs.append(lgbm_acc)

    # XGBoost
    xgb_model = XGBClassifier(eval_metric='logloss', random_state=12)
    xgb_model.fit(X_train, y_train)
    xgb_pred = xgb_model.predict(X_test)
    xgb_proba = xgb_model.predict_proba(X_test)[:, 1]
    xgb_auc = roc_auc_score(y_test, xgb_proba)
    xgb_f1 = f1_score(y_test, xgb_pred)
    xgb_acc = accuracy_score(y_test, xgb_pred)
    xgb_aucs.append(xgb_auc)
    xgb_f1s.append(xgb_f1)
    xgb_accs.append(xgb_acc)

# --- Results ---
print("LightGBM K-Fold:")
print("ROC AUCs:", np.round(lgbm_aucs, 4))
print("F1s:", np.round(lgbm_f1s, 4))
print("Accuracies:", np.round(lgbm_accs, 4))
print(f"Mean AUC: {np.mean(lgbm_aucs):.4f} | Std Dev: {np.std(lgbm_aucs):.4f}")
print(f"Mean F1: {np.mean(lgbm_f1s):.4f}")
print(f"Mean Accuracy: {np.mean(lgbm_accs):.4f}\n")

print("XGBoost K-Fold:")
print("ROC AUCs:", np.round(xgb_aucs, 4))
print("F1s:", np.round(xgb_f1s, 4))
print("Accuracies:", np.round(xgb_accs, 4))
print(f"Mean AUC: {np.mean(xgb_aucs):.4f} | Std Dev: {np.std(xgb_aucs):.4f}")
print(f"Mean F1: {np.mean(xgb_f1s):.4f}")
print(f"Mean Accuracy: {np.mean(xgb_accs):.4f}")

LightGBM K-Fold:
ROC AUCs: [0.9666 0.9681 0.9649 0.9638 0.9675]
F1s: [0.8982 0.9008 0.8954 0.8915 0.9051]
Accuracies: [0.8982 0.9012 0.8964 0.8916 0.9046]
Mean AUC: 0.9662 | Std Dev: 0.0016
Mean F1: 0.8982
Mean Accuracy: 0.8984

XGBoost K-Fold:
ROC AUCs: [0.9649 0.9687 0.9635 0.9639 0.9677]
F1s: [0.8972 0.9014 0.8937 0.8897 0.9023]
Accuracies: [0.8971 0.9019 0.8946 0.8902 0.9023]
Mean AUC: 0.9657 | Std Dev: 0.0021
Mean F1: 0.8969
Mean Accuracy: 0.8972


## 18. Shuffle Test Results for L1 & L2 Regularized Logistic Regression

The table below reports mean ROC AUC, F1 score, and accuracy for 10 shuffle runs.

In [20]:
n_runs = 10

l1_auc_shuff, l1_f1_shuff, l1_acc_shuff = [], [], []
l2_auc_shuff, l2_f1_shuff, l2_acc_shuff = [], [], []

for i in range(n_runs):
    # --- Shuffle labels independently ---
    _, y_shuffled = shuffle(X_scaled, y_full, random_state=12 + i)
    
    X_train, X_test, y_train_shuff, y_test_shuff = train_test_split(
        X_scaled, y_shuffled, test_size=0.3, stratify=y_shuffled, random_state=12 + i
    )
    
    # --- L1 Logistic Regression ---
    logreg_l1 = LogisticRegression(penalty='l1', solver='liblinear', max_iter=1000, random_state=12)
    logreg_l1.fit(X_train, y_train_shuff)
    y_pred_l1 = logreg_l1.predict(X_test)
    y_proba_l1 = logreg_l1.predict_proba(X_test)[:, 1]
    l1_auc_shuff.append(roc_auc_score(y_test_shuff, y_proba_l1))
    l1_f1_shuff.append(f1_score(y_test_shuff, y_pred_l1))
    l1_acc_shuff.append(accuracy_score(y_test_shuff, y_pred_l1))
    
    # --- L2 Logistic Regression ---
    logreg_l2 = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=1000, random_state=12)
    logreg_l2.fit(X_train, y_train_shuff)
    y_pred_l2 = logreg_l2.predict(X_test)
    y_proba_l2 = logreg_l2.predict_proba(X_test)[:, 1]
    l2_auc_shuff.append(roc_auc_score(y_test_shuff, y_proba_l2))
    l2_f1_shuff.append(f1_score(y_test_shuff, y_pred_l2))
    l2_acc_shuff.append(accuracy_score(y_test_shuff, y_pred_l2))

# --- Results ---
import numpy as np

print("L1 Logistic Regression (Shuffled Labels, 10 runs):")
print("AUCs:", np.round(l1_auc_shuff, 4))
print("F1s:", np.round(l1_f1_shuff, 4))
print("Accuracies:", np.round(l1_acc_shuff, 4))
print(f"Mean AUC: {np.mean(l1_auc_shuff):.4f} | Std Dev: {np.std(l1_auc_shuff):.4f}")
print(f"Mean F1: {np.mean(l1_f1_shuff):.4f}")
print(f"Mean Accuracy: {np.mean(l1_acc_shuff):.4f}\n")

print("L2 Logistic Regression (Shuffled Labels, 10 runs):")
print("AUCs:", np.round(l2_auc_shuff, 4))
print("F1s:", np.round(l2_f1_shuff, 4))
print("Accuracies:", np.round(l2_acc_shuff, 4))
print(f"Mean AUC: {np.mean(l2_auc_shuff):.4f} | Std Dev: {np.std(l2_auc_shuff):.4f}")
print(f"Mean F1: {np.mean(l2_f1_shuff):.4f}")
print(f"Mean Accuracy: {np.mean(l2_acc_shuff):.4f}")

L1 Logistic Regression (Shuffled Labels, 10 runs):
AUCs: [0.5109 0.5008 0.4944 0.505  0.4942 0.5019 0.51   0.4852 0.5063 0.4951]
F1s: [0.4986 0.4685 0.4853 0.5154 0.4887 0.4968 0.5077 0.4987 0.5055 0.4852]
Accuracies: [0.5092 0.5001 0.4957 0.5052 0.4923 0.5034 0.5097 0.485  0.5086 0.4935]
Mean AUC: 0.5004 | Std Dev: 0.0077
Mean F1: 0.4950
Mean Accuracy: 0.5003

L2 Logistic Regression (Shuffled Labels, 10 runs):
AUCs: [0.5109 0.5011 0.4948 0.5056 0.4946 0.5025 0.5107 0.4844 0.5062 0.4951]
F1s: [0.5012 0.4659 0.4859 0.5127 0.4869 0.4968 0.5084 0.4955 0.504  0.4858]
Accuracies: [0.5122 0.4981 0.4961 0.5046 0.4919 0.5016 0.5098 0.4823 0.5071 0.4951]
Mean AUC: 0.5006 | Std Dev: 0.0080
Mean F1: 0.4943
Mean Accuracy: 0.4999


## 19. K-Fold Cross-Validation Results for L1 & L2 Regularized Logistic Regression

The table below summarizes mean ROC AUC, F1 score, and accuracy across 5 folds for each model.

In [21]:
# --- Full dataset (already preprocessed into X and y) ---
X_full = X.copy()
y_full = y.copy()

# --- Scale full data ---
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_full)

# --- 5-Fold Stratified CV ---
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=12)

l1_aucs, l1_f1s, l1_accs = [], [], []
l2_aucs, l2_f1s, l2_accs = [], [], []

for train_index, test_index in kf.split(X_scaled, y_full):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y_full.iloc[train_index], y_full.iloc[test_index]

    # --- L1 Logistic Regression ---
    logreg_l1 = LogisticRegression(penalty='l1', solver='liblinear', max_iter=1000, random_state=12)
    logreg_l1.fit(X_train, y_train)
    y_pred_l1 = logreg_l1.predict(X_test)
    y_proba_l1 = logreg_l1.predict_proba(X_test)[:, 1]
    l1_aucs.append(roc_auc_score(y_test, y_proba_l1))
    l1_f1s.append(f1_score(y_test, y_pred_l1))
    l1_accs.append(accuracy_score(y_test, y_pred_l1))

    # --- L2 Logistic Regression ---
    logreg_l2 = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=1000, random_state=12)
    logreg_l2.fit(X_train, y_train)
    y_pred_l2 = logreg_l2.predict(X_test)
    y_proba_l2 = logreg_l2.predict_proba(X_test)[:, 1]
    l2_aucs.append(roc_auc_score(y_test, y_proba_l2))
    l2_f1s.append(f1_score(y_test, y_pred_l2))
    l2_accs.append(accuracy_score(y_test, y_pred_l2))

# --- Results ---
print("L1 Logistic Regression (5-Fold CV):")
print("ROC AUCs:", np.round(l1_aucs, 4))
print("F1s:", np.round(l1_f1s, 4))
print("Accuracies:", np.round(l1_accs, 4))
print(f"Mean AUC: {np.mean(l1_aucs):.4f} | Std Dev: {np.std(l1_aucs):.4f}")
print(f"Mean F1: {np.mean(l1_f1s):.4f}")
print(f"Mean Accuracy: {np.mean(l1_accs):.4f}\n")

print("L2 Logistic Regression (5-Fold CV):")
print("ROC AUCs:", np.round(l2_aucs, 4))
print("F1s:", np.round(l2_f1s, 4))
print("Accuracies:", np.round(l2_accs, 4))
print(f"Mean AUC: {np.mean(l2_aucs):.4f} | Std Dev: {np.std(l2_aucs):.4f}")
print(f"Mean F1: {np.mean(l2_f1s):.4f}")
print(f"Mean Accuracy: {np.mean(l2_accs):.4f}")

L1 Logistic Regression (5-Fold CV):
ROC AUCs: [0.946  0.9514 0.9494 0.9431 0.9473]
F1s: [0.8826 0.8855 0.8781 0.8769 0.8865]
Accuracies: [0.8843 0.8873 0.8804 0.8797 0.888 ]
Mean AUC: 0.9474 | Std Dev: 0.0029
Mean F1: 0.8819
Mean Accuracy: 0.8839

L2 Logistic Regression (5-Fold CV):
ROC AUCs: [0.9459 0.9514 0.9493 0.943  0.9472]
F1s: [0.8832 0.8845 0.8784 0.8772 0.8868]
Accuracies: [0.885  0.8864 0.8806 0.88   0.8882]
Mean AUC: 0.9474 | Std Dev: 0.0029
Mean F1: 0.8820
Mean Accuracy: 0.8840


## 20. Shuffle Test Results for Tuned Neural Network

The table below reports mean ROC AUC, F1 score, and accuracy for 10 shuffle runs.

In [22]:
n_runs = 10

mlp_auc_shuff, mlp_f1_shuff, mlp_acc_shuff = [], [], []

for i in range(n_runs):
    _, y_shuffled = shuffle(X_scaled, y_full, random_state=12 + i)

    X_train, X_test, y_train_shuff, y_test_shuff = train_test_split(
        X_scaled, y_shuffled, test_size=0.3, stratify=y_shuffled, random_state=12 + i
    )

    mlp_clf = MLPClassifier(
        hidden_layer_sizes=(128, 64),
        activation='relu',
        solver='adam',
        max_iter=1000,
        random_state=12
    )
    mlp_clf.fit(X_train, y_train_shuff)
    y_pred_mlp = mlp_clf.predict(X_test)
    y_proba_mlp = mlp_clf.predict_proba(X_test)[:, 1]

    mlp_auc_shuff.append(roc_auc_score(y_test_shuff, y_proba_mlp))
    mlp_f1_shuff.append(f1_score(y_test_shuff, y_pred_mlp))
    mlp_acc_shuff.append(accuracy_score(y_test_shuff, y_pred_mlp))

# --- Results ---
print("MLP (Shuffled Labels, 10 runs):")
print("AUCs:", np.round(mlp_auc_shuff, 4))
print("F1s:", np.round(mlp_f1_shuff, 4))
print("Accuracies:", np.round(mlp_acc_shuff, 4))
print(f"Mean AUC: {np.mean(mlp_auc_shuff):.4f} | Std Dev: {np.std(mlp_auc_shuff):.4f}")
print(f"Mean F1: {np.mean(mlp_f1_shuff):.4f}")
print(f"Mean Accuracy: {np.mean(mlp_acc_shuff):.4f}")

MLP (Shuffled Labels, 10 runs):
AUCs: [0.4973 0.5048 0.5029 0.4899 0.4958 0.5042 0.5033 0.491  0.4994 0.4959]
F1s: [0.5098 0.4969 0.4932 0.4805 0.5145 0.499  0.4815 0.4962 0.5114 0.5024]
Accuracies: [0.4941 0.5022 0.4966 0.4902 0.497  0.4984 0.499  0.4916 0.4999 0.494 ]
Mean AUC: 0.4985 | Std Dev: 0.0051
Mean F1: 0.4986
Mean Accuracy: 0.4963


## 21. K-Fold Cross-Validation Results for Tuned Neural Network

The table below summarizes mean ROC AUC, F1 score, and accuracy across 5 folds for the tuned MLP model.

In [23]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=12)

mlp_aucs, mlp_f1s, mlp_accs = [], [], []

for train_index, test_index in kf.split(X_scaled, y_full):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y_full.iloc[train_index], y_full.iloc[test_index]

    mlp_clf = MLPClassifier(
        hidden_layer_sizes=(128, 64),
        activation='relu',
        solver='adam',
        max_iter=1000,
        random_state=12
    )
    mlp_clf.fit(X_train, y_train)
    y_pred_mlp = mlp_clf.predict(X_test)
    y_proba_mlp = mlp_clf.predict_proba(X_test)[:, 1]

    mlp_aucs.append(roc_auc_score(y_test, y_proba_mlp))
    mlp_f1s.append(f1_score(y_test, y_pred_mlp))
    mlp_accs.append(accuracy_score(y_test, y_pred_mlp))

# --- Results ---
print("MLP (5-Fold CV):")
print("ROC AUCs:", np.round(mlp_aucs, 4))
print("F1s:", np.round(mlp_f1s, 4))
print("Accuracies:", np.round(mlp_accs, 4))
print(f"Mean AUC: {np.mean(mlp_aucs):.4f} | Std Dev: {np.std(mlp_aucs):.4f}")
print(f"Mean F1: {np.mean(mlp_f1s):.4f}")
print(f"Mean Accuracy: {np.mean(mlp_accs):.4f}")

MLP (5-Fold CV):
ROC AUCs: [0.9429 0.9504 0.9449 0.9504 0.9449]
F1s: [0.8756 0.8783 0.8722 0.8792 0.8784]
Accuracies: [0.8749 0.8797 0.8743 0.8795 0.8763]
Mean AUC: 0.9467 | Std Dev: 0.0031
Mean F1: 0.8767
Mean Accuracy: 0.8770
